We have PySpark DataFrame which containing log message from various applications.Each log line includes: app_name,log_level(info,error,debug, etc.)
log message(free text),timestamp.

Our task is to count how many logs per application contain keywords like "timeout","failed", or "exception"(case-insentive).
Group By app and keyword

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lower,explode,array,lit,when,split
from pyspark.sql.types import StructType,StructField,StringType,TimestampType

In [34]:
spark=SparkSession.builder.appName("log_message").getOrCreate()

In [35]:
data=[
 ("auth", "ERROR", "Login failed due to error", "2024-03-28 10:00:00"),("payment", "INFO", "Payment timeout occurred", "2024-03-28 10:01:00"),("auth", "INFO", "User logged in", "2024-03-28 10:02:00"),("payment", "ERROR", "Unknown exception raised", "2024-03-28 10:03:00"),
 ("orders", "INFO", "Order placed successfully", "2024-03-28 10:04:00")
 ]


In [36]:
data

[('auth', 'ERROR', 'Login failed due to error', '2024-03-28 10:00:00'),
 ('payment', 'INFO', 'Payment timeout occurred', '2024-03-28 10:01:00'),
 ('auth', 'INFO', 'User logged in', '2024-03-28 10:02:00'),
 ('payment', 'ERROR', 'Unknown exception raised', '2024-03-28 10:03:00'),
 ('orders', 'INFO', 'Order placed successfully', '2024-03-28 10:04:00')]

In [37]:
schema=StructType([
    StructField("app_name",StringType()),
    StructField("level",StringType()),
    StructField("log_message",StringType()),
    StructField("Timestamp",StringType())
])
schema

StructType([StructField('app_name', StringType(), True), StructField('level', StringType(), True), StructField('log_message', StringType(), True), StructField('Timestamp', StringType(), True)])

In [38]:
df=spark.createDataFrame(data,schema).withColumn("timestamp",col("timestamp").cast(TimestampType())).withColumn("log_mesage",lower(col("log_message")))

In [39]:
keywords=["timeout","failed","exception"]

Create array of matched keywords

In [49]:
matches=[
    when(col("log_message").contains(k),lit(k)) for k in keywords
    ]
df=df.withColumn("log_message",lower(col("log_message")))

In [50]:
df.printSchema()

root
 |-- app_name: string (nullable = true)
 |-- level: string (nullable = true)
 |-- log_message: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- log_mesage: string (nullable = true)
 |-- keyword: string (nullable = false)



Split,Explode and filter non-null

In [43]:
df = df.withColumn("keyword", split(col("log_message"), " ")) \
       .withColumn("keyword", explode(col("keyword"))) \
       .filter(col("keyword").isNotNull())

Group and Count

In [46]:
result=df.groupBy("app_name","log_message","keyword").count()

In [56]:
result=df.groupBy("app_name","log_message","keyword","Timestamp").count()
result

DataFrame[app_name: string, log_message: string, keyword: string, Timestamp: timestamp, count: bigint]

In [57]:

result.show()

+--------+--------------------+-------+-------------------+-----+
|app_name|         log_message|keyword|          Timestamp|count|
+--------+--------------------+-------+-------------------+-----+
| payment|payment timeout o...|      e|2024-03-28 10:01:00|    9|
| payment|payment timeout o...|      i|2024-03-28 10:01:00|    3|
|    auth|login failed due ...|      d|2024-03-28 10:00:00|   10|
| payment|payment timeout o...|      m|2024-03-28 10:01:00|    6|
|    auth|login failed due ...|      e|2024-03-28 10:00:00|   15|
|    auth|login failed due ...|      f|2024-03-28 10:00:00|    5|
| payment|payment timeout o...|      p|2024-03-28 10:01:00|    3|
|    auth|login failed due ...|      l|2024-03-28 10:00:00|   10|
|    auth|login failed due ...|      o|2024-03-28 10:00:00|   15|
|    auth|login failed due ...|      a|2024-03-28 10:00:00|    5|
| payment|payment timeout o...|      c|2024-03-28 10:01:00|    6|
|    auth|login failed due ...|       |2024-03-28 10:00:00|   20|
|    auth|